In [1]:
import os
import cv2
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from utils import inv2depth, transform, transform_midas
from depth_resnet import DepthResNet

In [2]:
basedir = 'datasets/3rdfloor'

In [3]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
device = torch.device('cpu')

In [ ]:
# torch.cuda.empty_cache()
# device = torch.device('cuda')

In [4]:
model = torch.hub.load("intel-isl/MiDaS", "MiDaS")
tr = transform_midas()

Downloading: "https://github.com/intel-isl/MiDaS/archive/master.zip" to /Users/blob/.cache/torch/hub/master.zip


Loading weights:  None


Downloading: "https://github.com/facebookresearch/WSL-Images/archive/master.zip" to /Users/blob/.cache/torch/hub/master.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /Users/blob/.cache/torch/hub/checkpoints/ig_resnext101_32x8-c38310e5.pth


  0%|          | 0.00/340M [00:00<?, ?B/s]

Downloading: "https://github.com/intel-isl/MiDaS/releases/download/v2_1/model-f6b98070.pt" to /Users/blob/.cache/torch/hub/checkpoints/model-f6b98070.pt


  0%|          | 0.00/403M [00:00<?, ?B/s]

In [4]:
# model = DepthResNet(version='50pt')
# checkpoint_path = "checkpoint_3rdfloor.pth"
# checkpoint = torch.load(checkpoint_path)
# state_dict = checkpoint['model']
# model.load_state_dict(state_dict)
# tr = transform()

In [5]:
model = model.to(device)
model.eval()
image_dir = os.path.join(basedir, 'images')
result_dir = os.path.join(basedir, 'depth_midas')
os.mkdir(result_dir)

In [6]:
# write midas
for imname in tqdm(os.listdir(image_dir)):
    img = Image.open(os.path.join(image_dir, imname))
    W, H = img.size
    img_tfm = tr(img).to(device)
    with torch.no_grad():
        inv_depth = model(img_tfm.unsqueeze(0))
    inv_depth = nn.functional.interpolate(inv_depth.unsqueeze(1),
                                          (H, W), mode='bilinear', align_corners=True)
    inv_depth = inv_depth.squeeze().cpu().numpy()
    depth_path = os.path.join(result_dir, imname.replace('.jpg', '.png'))
    cv2.imwrite(depth_path, inv_depth.astype('uint16'))

100%|██████████| 825/825 [1:16:49<00:00,  5.59s/it]


In [7]:
# write mine
# for imname in tqdm(os.listdir(image_dir)):
#     img = Image.open(os.path.join(image_dir, imname))
#     W, H = img.size
#     img_tfm = tr(img).to(device)
#     with torch.no_grad():
#         inv_depth = model(img_tfm.unsqueeze(0))
#     depth = inv2depth(inv_depth)
#     depth = nn.functional.interpolate(depth, (H, W), mode='bilinear', align_corners=True)
#     depth = depth.squeeze().cpu().numpy()
#     depth_path = os.path.join(result_dir, imname.replace('.jpg', '.png'))
#     cv2.imwrite(depth_path, (depth * 1000).astype('uint16'))

100%|██████████| 825/825 [01:17<00:00, 10.59it/s]
